# VacationPy


In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [ ]:
output_data_file = "output_data/cities.csv"
city_df = pd.read_csv(output_data_file)
city_df.head()

### Humidity Heatmap

In [ ]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [ ]:
# Bring in location data pairs, 
locations = city_df[["Lat", "Lng"]]

#humidity data, 
humidity = city_df['Humidity']

#define map 
fig = gmaps.figure()

#and layer, and add layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

### Create new DataFrame fitting weather criteria
The dataset is now narrowed down to suit my tastes. 
My ideal weather conditions:
####    A max temperature lower than 80 degrees but higher than 70.
####    Wind speed less than 10 mph.
####    Humidity between 30% and 60%.


In [ ]:
ideal_city_df = city_df[city_df['Cloudiness']<10]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp']>60]
ideal_city_df = ideal_city_df[ideal_city_df['Max Temp']<75]
ideal_city_df = ideal_city_df[ideal_city_df['Wind Speed']<20]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity']<60]
ideal_city_df = ideal_city_df[ideal_city_df['Humidity']>30]
ideal_city_df = ideal_city_df.dropna()
ideal_city_df


### Hotel Map
Now that I have my ideal vacation spot, I need a place to stay! The following code uses Google Maps API to find an appropriate hotel in each city.

In [ ]:
# Create dataframe and empty column
hotel_df = ideal_city_df[['City', 'Country','Lat', 'Lng']]
hotel_df = hotel_df.reindex(columns = ['Hotel Name','City', 'Country', 'Lat', 'Lng'])
hotel_df = hotel_df.reset_index(drop = True)

hotel_df


In [ ]:
#Use Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
target_type = "lodging"
radius = 5000
params = {
    "location": "",
    "type": target_type,
    "radius": radius,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(base_url, params).json()
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except:
        print("oops, something went wrong with the API call")
        #print(response)

hotel_df

#Plot the hotels on top of the humidity heatmap, with each pin containing the Hotel Name, City, and Country, as in the following image:


In [ ]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
# Create marker layer
marker_layer = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig

